# Escuela Politécnica Nacional

### Objetivos

Este proyecto tiene como objetivo medir el tiempo promedio que los estudiantes de la EPN que ingresarone el semestre 2012-A (pertenecientes a la primera promoción del SENECYT)se tardaron en culminar su carrera y encontrar si existe una relación entre la carrera y el tiempo de graduación.
Se debe tener en cuenta que como se trabaja con la lista de planes de tesis, los estudiantes considerados en el anális son solo los que se graduan mediante el proceso de titulación por proyecto.

Librerías utilizadas

In [5]:
import tabula
import pandas as pd
from datetime import datetime
import math

### Parámetros y configuraciones

In [6]:
FILE_PATH_STUDENT = "inputs/APROBADOS INGENIERIA_2012.pdf"
FILE_PATH_STUDENT_HTML = "inputs/planesTesis.html"
FILE_PATH_STUDENTS_CSV = "epnStudents2021.csv"

## Extracción

Para realizar este análisis, se toma información de dos fuentes: 
- Lista de estudiantes aprobados para inicar el prepólitecnico en el semestre 2012 - A en formato PDF 
-  Lista de todos los planes aprovados disponible en la página del saew [saew](https://saew.epn.edu.ec/SAETESIS/BusquedaPlanTesis.aspx) en formato HTML

### Extración de información de estudiantes aprovados para el 2012-A en formato PDF

Leer información del pdf y convertir en archivo CSV

In [7]:
def getInfoFromPdf():
    tabula.convert_into(FILE_PATH_STUDENT, 'epnStudents2021.csv', pages='all')

Método para generar el dataset a partir un CSV

In [8]:
def getDatasetFromCSVFile(file_path_param):
    data_set = pd.read_csv(FILE_PATH_STUDENTS_CSV)
    return data_set

Método para generar el correo electrónico de los estudiantes, este campo será utilizado para hacer match
con el otro dataset

In [9]:
def addEmail(df):
    names = df['NOMBRES']
    apellidos = df['APELLIDOS']

    # Generar los correos electrónicos
    correos = []
    for nombre, apellido in zip(names, apellidos):
        nombre_partes = nombre.lower().split()
        apellido_partes = apellido.lower().split()

        # Construir el correo electrónico
        correo = f"{nombre_partes[0]}.{apellido_partes[0]}@epn.edu.ec"
        correos.append(correo)

    df['Email'] = correos
    return df

Método que retorna un data set con la información <br>
[Nombre , Carrera, Email]

In [10]:
def getInfo2012Students():
    getInfoFromPdf()
    info_students_from_pdf = getDatasetFromCSVFile(FILE_PATH_STUDENTS_CSV)
    info_studens_with_email = addEmail(info_students_from_pdf)
    return info_studens_with_email
    

In [11]:
students_2012_info = getInfo2012Students()

In [12]:
#print(students_2012_info)

### Extración de información los planes de proyecto de titulación en formato HTML

Método para extraer información del archivo html, donde se encuentran los planes de tesis


In [13]:
def getInfoTesisFromHTML():
    dataframes = pd.read_html(FILE_PATH_STUDENT_HTML)

    columnEmail = dataframes[0]
    columnsName = ['Carrera','FecGradoOral','Email']
    new_emp_list = []

    for i in range(len(columnEmail)):
        carrera = columnEmail.iloc[i]['Carrera']
        fecGradoOral = columnEmail.iloc[i]['FecGradoOral']
        correos = columnEmail.iloc[i]['Email']
        for correo in correos.split():
            if "@epn.edu.ec" in correo:
                #print(correo)
                new_emp_list.append([carrera, fecGradoOral, correo])
    return pd.DataFrame(new_emp_list, columns=columnsName)

In [14]:
students_and_tesis = getInfoTesisFromHTML()

In [15]:
print(students_and_tesis)

                                                Carrera        FecGradoOral  \
0                    (RRA) ELECTRONICA Y AUTOMATIZACION                 NaN   
1                    (RRA) ELECTRONICA Y AUTOMATIZACION                 NaN   
2     (RRA) MAESTRIA DE INVESTIGACION EN GESTION DE ...                 NaN   
3     (RRA) MAESTRIA DE INVESTIGACION EN GESTION DE ...                 NaN   
4     (RRA) MAESTRIA DE INVESTIGACION EN GESTION DE ...                 NaN   
...                                                 ...                 ...   
8385             TECNOLOGIA EN MANTENIMIENTO INDUSTRIAL  17/08/2018 0:00:00   
8386             TECNOLOGIA EN MANTENIMIENTO INDUSTRIAL                 NaN   
8387             TECNOLOGIA EN MANTENIMIENTO INDUSTRIAL                 NaN   
8388      TECNOLOGIA EN PROCESOS DE PRODUCCION MECANICA  15/06/2017 0:00:00   
8389      TECNOLOGIA EN PROCESOS DE PRODUCCION MECANICA  15/06/2017 0:00:00   

                           Email  
0     juan.zambr

## Transformación

Para el proceso de transformación se procede a intersecar los dos dataframes, mediante el correo institucional

Método que genera un nuevo dataFrame realizando una intersección entre los dos dataFrames en base al campo email

In [16]:
def generar_dataframe_interseccion(df1, df2):
    # Obtiene los correos electrónicos únicos en ambos DataFrames
    emails_df1 = set(df1['Email'])
    emails_df2 = set(df2['Email'])
    
    
    # Encuentra los correos electrónicos que se repiten en ambos DataFrames
    emails_repetidos = list(emails_df1.intersection(emails_df2))
    
    # Filtra los DataFrames originales utilizando los correos electrónicos repetidos
    df1_interseccion = df1[df1['Email'].isin(emails_repetidos)]
    df2_interseccion = df2[df2['Email'].isin(emails_repetidos)]
    
    # Crea un nuevo DataFrame con los valores que se repiten en ambos DataFrames
    dataframe_interseccion = pd.concat([df1_interseccion, df2_interseccion], ignore_index=True)
    
    return dataframe_interseccion

In [17]:
merge_info = generar_dataframe_interseccion(students_2012_info,students_and_tesis)

Método que transforma la fecha

In [18]:
def convertir_fecha(cadena_fecha):
    formato = "%d/%m/%Y %H:%M:%S"
    fecha = datetime.strptime(cadena_fecha, formato)
    return fecha

Calular el tiempo

In [62]:
def restar_fechas(fecha1, fecha2):
    formato = '%Y-%m-%d %H:%M:%S'
    
    # Convertir las cadenas a objetos datetime
    fecha1_dt = datetime.strptime(fecha1, formato)
    fecha2_dt = datetime.strptime(fecha2, formato)
    
    # Calcular la diferencia de tiempo
    diferencia = fecha2_dt - fecha1_dt
    
    # Calcular la diferencia en años
    diferencia_anios = diferencia.days / 365.25
    
    # Retornar la diferencia en años
    return diferencia_anios


Método que toma le fecha en la que se registró la defensa de grado oral 'FecGradoOral' y se procede a calcular el tiempo transcurrido desde 2012


In [63]:
def calcular_anios(fecha1, fecha2):
    anios = fecha1.year - fecha2.year
    if fecha1.month < fecha2.month or (fecha1.month == fecha2.month and fecha1.day < fecha2.day):
        anios -= 1
    return anios

In [95]:
def add_elapsed_time_column(df):
    # Constantes
    formato = "%d/%m/%Y %H:%M:%S"
    date_time_2012 = datetime(2012, 1, 1, 0, 0, 0)
    
    oral_defense_date_string = df['FecGradoOral']
    elapsed_date_from_2012 = []
    
    for line in oral_defense_date_string:
        if isinstance(line, str):
            fecha_datetime = datetime.strptime(line, formato)
            elapsed_time = calcular_anios(fecha_datetime, date_time_2012)
            elapsed_date_from_2012.append(elapsed_time)
        else:
            elapsed_date_from_2012.append(0)
    
    df['TiempoGraduación'] = elapsed_date_from_2012
    return df
            

In [96]:
clean_info = add_elapsed_time_column(merge_info)

In [97]:
print(clean_info)

            APELLIDOS           NOMBRES  \
0       ABARCA ROBLES       JUAN DANIEL   
1     AGUILAR ALVAREZ    SANDRA NATHALY   
2    AGUILAR ENRÍQUEZ  MIGUEL ALEJANDRO   
3       AGUIRRE CUEVA  KLEBER ALEXANDER   
4         AGUIRRE IZA   STALIN PATRICIO   
..                ...               ...   
629               NaN               NaN   
630               NaN               NaN   
631               NaN               NaN   
632               NaN               NaN   
633               NaN               NaN   

                                               CARRERA  \
0                            INGENIERIA AGROINDUSTRIAL   
1        INGENIERIA ELECTRONICA Y REDES DE INFORMACION   
2                                           MATEMATICA   
3        INGENIERIA ELECTRONICA Y REDES DE INFORMACION   
4    INGENIERIA EN SISTEMAS INFORMATICOS Y DE COMPU...   
..                                                 ...   
629                                                NaN   
630                

## Load

Vamos a realizar un gráfico que nos indique que el timepo de la carrera

In [98]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
agrupado = clean_info.groupby(['CARRERA'])
print("***")
print(agrupado.get_group('INGENIERIA AGROINDUSTRIAL'))
print("---")
print(agrupado.get_group('INGENIERIA AGROINDUSTRIAL')['TiempoGraduación'].mean())
print("***")


***
              APELLIDOS             NOMBRES                    CARRERA  \
0         ABARCA ROBLES         JUAN DANIEL  INGENIERIA AGROINDUSTRIAL   
29        BRITO SEVILLA     DAYANNA LIZBETH  INGENIERIA AGROINDUSTRIAL   
54    CEVALLOS GALLEGOS         ALAN ANDRES  INGENIERIA AGROINDUSTRIAL   
66         CORNEJO PAEZ      CAMILA NATALIA  INGENIERIA AGROINDUSTRIAL   
74          CUESTA PLUA      MARIA CRISTINA  INGENIERIA AGROINDUSTRIAL   
80      DELGADO ALMEIDA    ANDREA ELIZABETH  INGENIERIA AGROINDUSTRIAL   
87        ESPINOZA BÁEZ  KATHERINE ANABELLE  INGENIERIA AGROINDUSTRIAL   
96    GALLEGOS CEVALLOS        CARLOS DARIO  INGENIERIA AGROINDUSTRIAL   
107    HERRERA QUIÑONEZ      TANIA CAROLINA  INGENIERIA AGROINDUSTRIAL   
133        LOZANO ORTIZ        BRYAN MIGUEL  INGENIERIA AGROINDUSTRIAL   
139     MARTINEZ CHULDE     KATHERINE ELISA  INGENIERIA AGROINDUSTRIAL   
192       PULLAS FLORES   ESTEFANI YESSENIA  INGENIERIA AGROINDUSTRIAL   
206      REVELO MORILLO  GABRIELA 